In [30]:
import pandas as pd

import os
import requests
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("RIOT_API_KEY")
BASE_URL = "https://americas.api.riotgames.com/lol/"

ACCOUNT_BASE_URL = (
    "https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/"
)


def get_account_info(game_name, tag_line):
    """
    Fetch account information given a game name and tag line.
    """
    url = f"{ACCOUNT_BASE_URL}{game_name}/{tag_line}"
    params = {"api_key": API_KEY}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()


def get_match_ids_by_puuid(puuid):
    """
    Fetch the list of match ids for the specified puuid.
    """
    url = f"{BASE_URL}match/v5/matches/by-puuid/{puuid}/ids"
    params = {"api_key": API_KEY, "start": 0, "count": 20}  # Adjust 'count' as needed
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()


# # Example usage
# if __name__ == "__main__":
#     game_name = "spiritedtraditin"  # Replace with your game name
#     tag_line = "unk"  # Replace with your tag line

#     try:
#         account_info = get_account_info(game_name, tag_line)
#         if account_info:
#             puuid = account_info.get("puuid")
#             match_ids = get_match_ids_by_puuid(puuid)
#             print(match_ids)
#         else:
#             print("Account not found.")
#     except Exception as e:
#         print(f"An error occurred: {e}")


In [31]:
game_name = "spiritedtraditin"  # Replace with your game name
tag_line = "unk"  # Replace with your tag line

try:
    account_info = get_account_info(game_name, tag_line)
    if account_info:
        puuid = account_info.get("puuid")
        match_ids = get_match_ids_by_puuid(puuid)
        print(match_ids)
    else:
        print("Account not found.")
except Exception as e:
    print(f"An error occurred: {e}")


['NA1_4896828903', 'NA1_4896795871', 'NA1_4896741877', 'NA1_4896677691', 'NA1_4896651121', 'NA1_4896608813', 'NA1_4896573457', 'NA1_4896557904', 'NA1_4896513807', 'NA1_4895570135', 'NA1_4895492392', 'NA1_4895460540', 'NA1_4894672305', 'NA1_4894619416', 'NA1_4894577850', 'NA1_4894543154', 'NA1_4893129550', 'NA1_4893019552', 'NA1_4892657141', 'NA1_4892625882']


In [32]:
match_ids

['NA1_4896828903',
 'NA1_4896795871',
 'NA1_4896741877',
 'NA1_4896677691',
 'NA1_4896651121',
 'NA1_4896608813',
 'NA1_4896573457',
 'NA1_4896557904',
 'NA1_4896513807',
 'NA1_4895570135',
 'NA1_4895492392',
 'NA1_4895460540',
 'NA1_4894672305',
 'NA1_4894619416',
 'NA1_4894577850',
 'NA1_4894543154',
 'NA1_4893129550',
 'NA1_4893019552',
 'NA1_4892657141',
 'NA1_4892625882']

In [67]:
MATCH_URL = "https://americas.api.riotgames.com/lol/match/v5/matches/"

def collect_match_data(match_ids):
    headers = {"X-Riot-Token": API_KEY}
    match_data = []

    for match_id in match_ids:
        response = requests.get(MATCH_URL + match_id, headers=headers)
        if response.status_code == 200:
            match_data.append(response.json())
        else:
            print(f"Failed to retrieve data for match ID {match_id}")

    df = pd.DataFrame(match_data)
    return df

In [66]:
MATCH_URL = "https://americas.api.riotgames.com/lol/match/v5/matches/"

def collect_match_data1(match_ids):
    API_KEY = os.getenv("RIOT_API_KEY")
    headers = {"X-Riot-Token": API_KEY}
    match_info = []
    team_info = []
    player_info = []

    for match_id in match_ids:
        response = requests.get(MATCH_URL + match_id, headers=headers)
        if response.status_code == 200:
            data = response.json()

            # Extracting match information
            match_details = {
                'Match ID': match_id,
                'Game Mode': data['info']['gameMode'],
                'Map ID': data['info']['mapId'],
                'Game Duration': data['info']['gameDuration'],
                'Game Version': data['info']['gameVersion']
            }
            match_info.append(match_details)

            # Extracting team information
            for team in data['info']['teams']:
                team_details = {
                    'Match ID': match_id,
                    'Team ID': team['teamId'],
                    'Win': team['win'],
                    'Inhibitors Lost': team.get('inhibitorKills', 0),
                    'Turrets Lost': team.get('towerKills', 0),
                    'Baron Kills': team.get('baronKills', 0),
                    'Dragon Kills': team.get('dragonKills', 0),
                    'Rift Herald Kills': team.get('riftHeraldKills', 0)
                }
                team_info.append(team_details)

            # Extracting player information
            for participant in data['info']['participants']:
                player_details = {
                    'Match ID': match_id,
                    'Player Name': participant['summonerName'],
                    'Champion': participant['championName'],
                    'Lane': participant['lane'],
                    'KDA': f"{participant['kills']}/{participant['deaths']}/{participant['assists']}",
                    'Gold Earned': participant['goldEarned'],
                    'Total Damage to Champions': participant['totalDamageDealtToChampions'],
                    'Vision Score': participant['visionScore']
                }
                player_info.append(player_details)
        else:
            print(f"Failed to retrieve data for match ID {match_id}")

    match_df = pd.DataFrame(match_info)
    team_df = pd.DataFrame(team_info)
    player_df = pd.DataFrame(player_info)

    return match_df, team_df, player_df


In [124]:
MATCH_URL = "https://americas.api.riotgames.com/lol/match/v5/matches/"

def collect_match_data2(match_ids):
    API_KEY = os.getenv("RIOT_API_KEY")
    headers = {"X-Riot-Token": API_KEY}
    match_info = []
    team_info = []
    player_info = []

    for match_id in match_ids:
        response = requests.get(MATCH_URL + match_id, headers=headers)
        if response.status_code == 200:
            data = response.json()

            # Extracting match information
            match_details = {
                'Match ID': match_id,
                'Game Mode': data['info']['gameMode'],
                'Map ID': data['info']['mapId'],
                'Game Duration': data['info']['gameDuration'],
                'Game Version': data['info']['gameVersion']
            }
            match_info.append(match_details)

            # Extracting team information
            for team in data['info']['teams']:
                team_details = {
                    'Match ID': match_id,
                    'Team ID': team['teamId'],
                    'Win': team['win'],
                    'Inhibitors Lost': team.get('inhibitorKills', 0),
                    'Turrets Lost': team.get('towerKills', 0),
                    'Baron Kills': team.get('baronKills', 0),
                    'Dragon Kills': team.get('dragonKills', 0),
                    'Rift Herald Kills': team.get('riftHeraldKills', 0)
                }
                team_info.append(team_details)

            # Extracting player information
            for participant in data['info']['participants']:
                player_details = {
                    'Match ID': match_id,
                    'Player Name': participant.get('summonerName', 'Unknown'),
                    'Riot ID Game Name': participant.get('riotIdGameName', 'Unknown'),  # Attempting to include riotIdGameName
                    'Champion': participant['championName'],
                    'Lane': participant['lane'],
                    'KDA': f"{participant['kills']}/{participant['deaths']}/{participant['assists']}",
                    'Gold Earned': participant['goldEarned'],
                    'Total Damage to Champions': participant['totalDamageDealtToChampions'],
                    'Vision Score': participant['visionScore']
                }
                player_info.append(player_details)
        else:
            print(f"Failed to retrieve data for match ID {match_id}")

    match_df = pd.DataFrame(match_info)
    team_df = pd.DataFrame(team_info)
    player_df = pd.DataFrame(player_info)

    return match_df, team_df, player_df

In [68]:
matches = collect_match_data(match_ids)

In [109]:
matche1 = collect_match_data1(match_ids)

In [125]:
matche2 = collect_match_data2(match_ids)

In [120]:
pd.DataFrame.from_dict(matches.loc[0]['info']['participants'])['riotIdGameName']

0            KevXin99
1        Redskinninja
2          Helmet Kid
3         Eribunny101
4      Manofthebutter
5         ChanChaChan
6              Shmoff
7        NautInMyYard
8       ThorThePriest
9    spiritedtraditin
Name: riotIdGameName, dtype: object

In [123]:
matche1[2]

,Match ID,Player Name,Champion,Lane,KDA,Gold Earned,Total Damage to Champions,Vision Score
0,NA1_4896828903,KevXin99,Teemo,JUNGLE,7/12/10,13890,21502,21
1,NA1_4896828903,AppleSniper87,Warwick,JUNGLE,10/11/9,14532,33974,24
2,NA1_4896828903,Helmet Kid,Akali,MIDDLE,22/10/2,17353,50389,9
3,NA1_4896828903,Eribunny101,Jinx,BOTTOM,14/10/7,14883,28965,36
4,NA1_4896828903,Manofthebutter,Seraphine,BOTTOM,1/6/17,9503,7426,31
...,...,...,...,...,...,...,...,...
195,NA1_4892625882,Ashe bot,Ashe,NONE,3/8/3,5452,8592,0
196,NA1_4892625882,Warwick bot,Warwick,NONE,1/11/0,4000,3621,0
197,NA1_4892625882,Ryze bot,Ryze,NONE,3/16/0,5244,4601,0
198,NA1_4892625882,Taric bot,Taric,NONE,1/9/2,4610,3637,0


In [129]:
matche2[2][matche2[2]['Riot ID Game Name']== 'spiritiedtraditin']

array(['KevXin99', 'Redskinninja', 'Helmet Kid', 'Eribunny101',
       'Manofthebutter', 'ChanChaChan', 'Shmoff', 'NautInMyYard',
       'ThorThePriest', 'spiritedtraditin', 'BSWAGS', 'whyaspork',
       'bearsecond', 'one', 'tzusHi', 'shika', 'lairitist', 'Ahsokam',
       'Devilishly', 'ShekelStealer', 'Rocboy', 'karateskis',
       'Furry Yuri', 'settussytroller', 'Ozzytroller', 'toxic mlm',
       'Kayns Mother', 'stedgee', 'harmleśś', 'beifong', 'globgrace',
       'Minh', 'GrimmyBoy', 'Minivanhero', 'Raindeer', 'Dyl Pickle',
       'Rohmean', 'RainyDay', 'AnonymousV', 'MusicalAuthor', 'ValleyGirl',
       'creepyking11', 'SeanBtheG21', 'sergentadoune', 'jer63y',
       'ThePinkShadow', 'simonak94', 'Wigglet Fess', 'meepjrjr',
       'Colby Brock', 'Sam Golbach', 'SpedyDee', 'drago', 'cryt',
       'FourthSpear', 'SPK1602', 'aimeem3ow', 'SipinTeaInYoHood', 'Arkzy',
       'I clean poo', 'femboy feet', 'dawnxxdusk', 'Anderson', 'Rogonsz',
       'Chris', 'Devon', 'RunItDownMid', 'C